In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import yaml
import pandas as pd
import numpy as np
from functools import partial
import multiprocessing
import tensorflow as tf
from pathlib import Path
from time import strftime
from shutil import rmtree
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib
matplotlib.use("TKAgg", force=True)
%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
from rtapipe.lib.utils.misc import dotdict
from rtapipe.lib.dataset.data_manager import DataManager
from rtapipe.lib.datasource.Photometry3 import OnlinePhotometry, SimulationParams
from rtapipe.lib.plotting.plotting import plot_sequences
from rtapipe.lib.models.anomaly_detector_builder import AnomalyDetectorBuilder
from rtapipe.lib.evaluation.pval import get_pval_table, get_threshold_for_sigma
from rtapipe.lib.standardanalysis.li_ma import LiMa
from rtapipe.lib.evaluation.pval import get_pval_table, get_threshold_for_sigma, get_sigma_from_ts, get_sigma_from_pvalue, get_sigma_for_ts_array

# Parameters

In [4]:
INTEGRATION_TIME=5
SCALER="robust"

# Loading the models

In [5]:
with open("./trained_models.yaml", "r") as f:
    try:
        configs = yaml.safe_load(f)
    except yaml.YAMLError as exc:
        print(exc)
configs = [dotdict(c) for c in configs["models"]]

In [5]:
for i, config in enumerate(configs):
    if      config.name     == "AnomalyDetector_cnn_l2_u32" \
        and config.patience == 5 \
        and config.integrationtime == INTEGRATION_TIME \
        and config.scaler == SCALER:
        
        break
model_config_cnn = configs[i]
model_config_cnn.ad = AnomalyDetectorBuilder.getAnomalyDetector(name=model_config_cnn.name, timesteps=model_config_cnn.timesteps, nfeatures=model_config_cnn.nfeatures, load_model="True", training_epoch_dir=model_config_cnn.path, training=False)
model_config_cnn.pvalue_table = get_pval_table(model_config_cnn.pval_path) 
model_config_cnn.name

AnomalyDetector_cnn_l2_u32 - input shape: (5,3)


'AnomalyDetector_cnn_l2_u32'

In [6]:
for i, config in enumerate(configs):
    if      config.name     == "AnomalyDetector_rnn_l2_u32" \
        and config.patience == 5 \
        and config.integrationtime == INTEGRATION_TIME \
        and config.scaler == SCALER:
        
        break
model_config_rnn = configs[i]
model_config_rnn.ad = AnomalyDetectorBuilder.getAnomalyDetector(name=model_config_rnn.name, timesteps=model_config_rnn.timesteps, nfeatures=model_config_rnn.nfeatures, load_model="True", training_epoch_dir=model_config_rnn.path, training=False)
model_config_rnn.pvalue_table = get_pval_table(model_config_rnn.pval_path) 
model_config_rnn.name

AnomalyDetector_rnn_l2_u32 - input shape: (5,3)


'AnomalyDetector_rnn_l2_u32'

# Loading the test dataset

In [7]:
output_dir = "./logs/test_models_dataset_itime_5_d_out"

In [8]:
features_names = ["EB_0.04-0.117","EB_2-0.117-0.342","EB_0.342-1"]

In [9]:
data_manager = DataManager(output_dir)

In [10]:
dataset_id="test_itime_5_d_tsl_5_nbins_3"

In [11]:
dataset_folder = "/data01/homes/baroncelli/phd/rtapipe/scripts/ml/dataset_generation/test/itime_5_d/fits_data"
fits_files = DataManager.load_fits_data(dataset_folder, limit=250)
test_set_size = len(fits_files)

Loaded 231 files


In [12]:
sim_params = SimulationParams(runid=None, onset=250, emin=0.04, emax=1, tmin=0, tobs=500, offset=0.5, irf="North_z40_5h_LST", roi=2.5, caldb="prod5-v0.1", simtype="grb")
multiple_templates = True
add_target_region = True
integration_time = 5
number_of_energy_bins = 3
tsl = 100
threads = 30
normalize = True
#data_manager.transform_to_timeseries(fits_files, sim_params, add_target_region, integration_time=integration_time, number_of_energy_bins=number_of_energy_bins, tsl=tsl, normalize=normalize, threads=threads, multiple_templates=multiple_templates)
data_manager.load_saved_data(5, 100)

Loading cached data from run0005_ID000225_it_5_tsl_100.npy
Loading cached data from run0009_ID000191_it_5_tsl_100.npy
Loading cached data from run0013_ID000321_it_5_tsl_100.npy
Loading cached data from run0017_ID000302_it_5_tsl_100.npy
Loading cached data from run0019_ID000338_it_5_tsl_100.npy
Loading cached data from run0021_ID000373_it_5_tsl_100.npy
Loading cached data from run0023_ID000267_it_5_tsl_100.npy
Loading cached data from run0025_ID000471_it_5_tsl_100.npy
Loading cached data from run0027_ID000288_it_5_tsl_100.npy
Loading cached data from run0036_ID000168_it_5_tsl_100.npy
Loading cached data from run0037_ID000283_it_5_tsl_100.npy
Loading cached data from run0039_ID000031_it_5_tsl_100.npy
Loading cached data from run0045_ID000095_it_5_tsl_100.npy
Loading cached data from run0048_ID000219_it_5_tsl_100.npy
Loading cached data from run0050_ID000319_it_5_tsl_100.npy
Loading cached data from run0053_ID000030_it_5_tsl_100.npy
Loading cached data from run0058_ID000423_it_5_tsl_100.n

In [13]:
assert len(data_manager.data) == test_set_size

## Plot some samples

In [ ]:
for template in list(data_manager.data.keys())[0:5]:
    data_manager.plot_timeseries(template, data_manager.data[template], 1, sim_params, output_dir, labels=features_names)

# TESTING

In [14]:
data_manager.load_scaler(model_config_rnn.scaler_path) # model_config_rnn.scaler_path or model_config_cnn.scaler_path

In [15]:
test_all_x, test_all_y = data_manager.get_test_set_all_templates(verbose=False, onset=250, integration_time=integration_time, sub_window_size=5, stride=1)

[2022-11-30 10:22:14.641886] Total x shape shape: (22176, 5, 3)
[2022-11-30 10:22:14.642029] Total y shape shape: (22176,)


# RNN

## Standard metrics - 3 sigma threshold

In [16]:
SIGMA_THRESHOLD = 3

In [18]:
sigma_threshold = get_threshold_for_sigma(model_config_rnn.pvalue_table, SIGMA_THRESHOLD)
model_config_rnn.ad.threshold = sigma_threshold
print(f"Threshold: {model_config_rnn.ad.threshold} corresponding to {get_sigma_from_ts(model_config_rnn.pvalue_table, model_config_rnn.ad.threshold)} sigma")

Threshold: 0.0005921241825 corresponding to 3.0060572682 sigma


In [19]:
print(f"************** Evaluating {model_config_rnn.name} patience={model_config_rnn.patience} **************")
metrics = model_config_rnn.ad.evaluate(test_all_x, test_all_y)
for k,v in metrics.items():
    print(k,v)
print("detection_delay:",model_config_rnn.ad.detection_delay(test_all_y, model_config_rnn.ad.predict(test_all_x), test_set_size, model_config_rnn.timesteps))

************** Evaluating AnomalyDetector_rnn_l2_u32 patience=5 **************
Using threshold 0.0005921241825
accuracy 0.5080266955266955
precision 0.9880668257756563
recall 0.036575669228730456
f1 0.07054012608621571
roc_auc 0.5180575684266523
confusion_matrix [[10852, 5], [10905, 414]]
false_positive_rate 0.011933174224343675
detection_delay: 5.688311688311688


## Standard metrics - 5 sigma threshold

In [20]:
SIGMA_THRESHOLD = 5

In [21]:
sigma_threshold = get_threshold_for_sigma(model_config_rnn.pvalue_table, SIGMA_THRESHOLD)
model_config_rnn.ad.threshold = sigma_threshold
print(f"Threshold: {model_config_rnn.ad.threshold} corresponding to {get_sigma_from_ts(model_config_rnn.pvalue_table, model_config_rnn.ad.threshold)} sigma")

Threshold: 0.0020782086315 corresponding to 5.0488536162 sigma


In [22]:
print(f"************** Evaluating {model_config_rnn.name} patience={model_config_rnn.patience} **************")
metrics = model_config_rnn.ad.evaluate(test_all_x, test_all_y)
for k,v in metrics.items():
    print(k,v)
print("detection_delay:",model_config_rnn.ad.detection_delay(test_all_y, model_config_rnn.ad.predict(test_all_x), test_set_size, model_config_rnn.timesteps))

************** Evaluating AnomalyDetector_rnn_l2_u32 patience=5 **************
Using threshold 0.0020782086315
accuracy 0.4902597402597403
precision 0.8571428571428571
recall 0.0015902464882056719
f1 0.0031746031746031746
roc_auc 0.5006569635314749
confusion_matrix [[10854, 3], [11301, 18]]
false_positive_rate 0.14285714285714285
detection_delay: 0.2727272727272727


## Sigma table

In [23]:
data = {}
for template in tqdm(list(data_manager.data.keys())):
    test_x, test_y = data_manager.get_test_set(template=template, onset=250, integration_time=5, sub_window_size=5, stride=1)
    model_config_rnn.ad.predict(test_x)
    data[template] = get_sigma_for_ts_array(model_config_rnn.pvalue_table, model_config_rnn.ad.get_reconstruction_errors(), verbose=False)

sigma_table_rnn = pd.DataFrame(data, index=DataManager.get_tbin_of_sequences(96, 5, 5, 1))    

100%|█████████████████████████████████████████| 231/231 [00:13<00:00, 16.59it/s]


In [24]:
sigma_table_rnn

,run0005_ID000225,run0009_ID000191,run0013_ID000321,run0017_ID000302,run0019_ID000338,run0021_ID000373,run0023_ID000267,run0025_ID000471,run0027_ID000288,run0036_ID000168,run0037_ID000283,run0039_ID000031,run0045_ID000095,run0048_ID000219,run0050_ID000319,run0053_ID000030,run0058_ID000423,run0066_ID000123,run0067_ID000038,run0074_ID000249,run0077_ID000381,run0079_ID000181,run0080_ID000283,run0087_ID000156,run0091_ID000232,run0103_ID000247,run0106_ID000232,run0108_ID000020,run0111_ID000118,run0117_ID000327,run0121_ID000209,run0122_ID000021,run0124_ID000071,run0125_ID000077,run0126_ID000344,run0131_ID000391,run0132_ID000380,run0150_ID000189,run0151_ID000312,run0156_ID000028,run0158_ID000359,run0159_ID000035,run0160_ID000149,run0162_ID000268,run0167_ID000041,run0170_ID000124,run0170_ID000218,run0170_ID000400,run0170_ID000418,run0175_ID000139,run0175_ID000300,run0176_ID000218,run0176_ID000272,run0177_ID000070,run0183_ID000331,run0187_ID000435,run0189_ID000293,run0191_ID000229,run0193_ID000013,run0195_ID000111,run0202_ID000245,run0204_ID000284,run0207_ID000154,run0209_ID000300,run0210_ID000016,run0210_ID000040,run0210_ID000221,run0211_ID000312,run0211_ID000462,run0216_ID000185,run0216_ID000333,run0219_ID000029,run0221_ID000373,run0222_ID000126,run0226_ID000442,run0228_ID000404,run0236_ID000181,run0240_ID000477,run0242_ID000208,run0245_ID000236,run0250_ID000440,run0251_ID000377,run0254_ID000090,run0254_ID000470,run0256_ID000112,run0263_ID000260,run0266_ID000397,run0270_ID000260,run0270_ID000262,run0271_ID000336,run0276_ID000003,run0276_ID000042,run0278_ID000111,run0279_ID000405,run0280_ID000101,run0285_ID000014,run0286_ID000206,run0290_ID000056,run0291_ID000193,run0292_ID000156,run0298_ID000339,run0302_ID000382,run0314_ID000223,run0321_ID000159,run0323_ID000052,run0339_ID000132,run0340_ID000247,run0342_ID000307,run0347_ID000361,run0350_ID000115,run0352_ID000021,run0352_ID000163,run0355_ID000275,run0356_ID000097,run0361_ID000216,run0368_ID000124,run0374_ID000020,run0378_ID000129,run0384_ID000378,run0389_ID000142,run0389_ID000280,run0396_ID000359,run0401_ID000227,run0410_ID000250,run0410_ID000273,run0415_ID000055,run0419_ID000300,run0420_ID000152,run0421_ID000208,run0421_ID000383,run0434_ID000189,run0445_ID000162,run0446_ID000308,run0454_ID000028,run0468_ID000072,run0473_ID000322,run0475_ID000120,run0481_ID000069,run0484_ID000321,run0486_ID000108,run0487_ID000320,run0489_ID000040,run0489_ID000339,run0489_ID000353,run0491_ID000173,run0492_ID000231,run0498_ID000168,run0503_ID000143,run0509_ID000243,run0512_ID000238,run0516_ID000336,run0522_ID000294,run0524_ID000087,run0529_ID000420,run0533_ID000144,run0535_ID000141,run0537_ID000149,run0543_ID000437,run0546_ID000406,run0550_ID000253,run0550_ID000494,run0551_ID000099,run0551_ID000150,run0557_ID000319,run0558_ID000096,run0564_ID000354,run0577_ID000106,run0593_ID000019,run0598_ID000199,run0603_ID000005,run0603_ID000195,run0606_ID000304,run0608_ID000286,run0610_ID000063,run0613_ID000309,run0613_ID000375,run0615_ID000245,run0617_ID000297,run0619_ID000308,run0627_ID000386,run0629_ID000087,run0635_ID000075,run0642_ID000185,run0642_ID000462,run0647_ID000398,run0653_ID000037,run0653_ID000078,run0658_ID000054,run0664_ID000064,run0664_ID000350,run0667_ID000302,run0668_ID000294,run0672_ID000265,run0677_ID000457,run0684_ID000011,run0684_ID000346,run0687_ID000058,run0688_ID000432,run0689_ID000454,run0689_ID000513,run0700_ID000206,run0710_ID000198,run0719_ID000018,run0720_ID000203,run0729_ID000124,run0731_ID000202,run0734_ID000309,run0735_ID000071,run0766_ID000057,run0782_ID000343,run0785_ID000315,run0787_ID000040,run0793_ID000454,run0798_ID000437,run0804_ID000264,run0808_ID000191,run0813_ID000124,run0822_ID000056,run0836_ID000198,run0836_ID000340,run0837_ID000263,run0842_ID000055,run0842_ID000212,run0842_ID000312,run0865_ID000336,run0869_ID000370,run0877_ID000060,run0877_ID000233,run0885_ID000163,run0889_ID000211,run0889_ID000364
0-25,0.000000,0.000000,0,0,0.000000,0,0.000000,0,0,0,0.000000,0,0,0.0000

# CNN

## Standard metrics - 3 sigma

In [25]:
SIGMA_THRESHOLD = 3

In [26]:
sigma_threshold = get_threshold_for_sigma(model_config_cnn.pvalue_table, SIGMA_THRESHOLD)
model_config_cnn.ad.threshold = sigma_threshold
print(f"Threshold: {model_config_cnn.ad.threshold} corresponding to {get_sigma_from_ts(model_config_cnn.pvalue_table, model_config_cnn.ad.threshold)} sigma")

Threshold: 0.006506945827 corresponding to 3.0286528291 sigma


In [27]:
print(f"************** Evaluating {model_config_cnn.name} patience={model_config_cnn.patience} **************")
metrics = model_config_cnn.ad.evaluate(test_all_x, test_all_y)
for k,v in metrics.items():
    print(k,v)
print("detection_delay:",model_config_cnn.ad.detection_delay(test_all_y, model_config_cnn.ad.predict(test_all_x), test_set_size, model_config_cnn.timesteps))

************** Evaluating AnomalyDetector_cnn_l2_u32 patience=5 **************
Using threshold 0.006506945827
accuracy 0.4910263347763348
precision 0.9210526315789473
recall 0.0030921459492888066
f1 0.006163599542132606
roc_auc 0.5014079132620165
confusion_matrix [[10854, 3], [11284, 35]]
false_positive_rate 0.07894736842105263
detection_delay: 1.0


## Standard metrics - 5 sigma

In [28]:
SIGMA_THRESHOLD = 5

In [29]:
sigma_threshold = get_threshold_for_sigma(model_config_cnn.pvalue_table, SIGMA_THRESHOLD)
model_config_cnn.ad.threshold = sigma_threshold
print(f"Threshold: {model_config_cnn.ad.threshold} corresponding to {get_sigma_from_ts(model_config_cnn.pvalue_table, model_config_cnn.ad.threshold)} sigma")

Threshold: 0.0139048776129999 corresponding to 5.0465140766 sigma


In [30]:
print(f"************** Evaluating {model_config_cnn.name} patience={model_config_cnn.patience} **************")
metrics = model_config_cnn.ad.evaluate(test_all_x, test_all_y)
for k,v in metrics.items():
    print(k,v)
print("detection_delay:",model_config_cnn.ad.detection_delay(test_all_y, model_config_cnn.ad.predict(test_all_x), test_set_size, model_config_cnn.timesteps))

************** Evaluating AnomalyDetector_cnn_l2_u32 patience=5 **************
Using threshold 0.0139048776129999
accuracy 0.48976370851370854
precision 1.0
recall 0.0003533881084901493
f1 0.0007065265389031176
roc_auc 0.500176694054245
confusion_matrix [[10857, 0], [11315, 4]]
false_positive_rate 0.0
detection_delay: 0.030303030303030304


## Sigma table

In [31]:
data = {}
for template in tqdm(list(data_manager.data.keys())):
    test_x, test_y = data_manager.get_test_set(template=template, onset=250, integration_time=5, sub_window_size=5, stride=1)
    model_config_cnn.ad.predict(test_x)
    data[template] = get_sigma_for_ts_array(model_config_cnn.pvalue_table, model_config_cnn.ad.get_reconstruction_errors(), verbose=False)

sigma_table_cnn = pd.DataFrame(data, index=DataManager.get_tbin_of_sequences(96, 5, 5, 1))    

100%|█████████████████████████████████████████| 231/231 [00:13<00:00, 17.13it/s]


# Li&Ma

In [32]:
li_ma_data = {}
templates = list(data_manager.data.keys())
templates = []
for template in tqdm(templates):
    lima_df = LiMa.detect("binned", DataManager.get_fits_from_template(fits_files, template), temporal_bins=sigma_table_cnn.index.values, tobs=500, sigma_gt=0)
    li_ma_data[template] = lima_df["sigma"].values
lima_sigma_table = pd.DataFrame(li_ma_data, index=sigma_table_cnn.index.values)
if len(templates) > 0:
    lima_sigma_table.to_csv(Path(output_dir).joinpath("li_ma_detection_table.csv"))    

0it [00:00, ?it/s]


In [33]:
lima_sigma_table = pd.read_csv(Path(output_dir).joinpath("li_ma_detection_table.csv"), index_col=0)

In [34]:
lima_sigma_table

,run0005_ID000225,run0009_ID000191,run0013_ID000321,run0017_ID000302,run0019_ID000338,run0021_ID000373,run0023_ID000267,run0025_ID000471,run0027_ID000288,run0036_ID000168,run0037_ID000283,run0039_ID000031,run0045_ID000095,run0048_ID000219,run0050_ID000319,run0053_ID000030,run0058_ID000423,run0066_ID000123,run0067_ID000038,run0074_ID000249,run0077_ID000381,run0079_ID000181,run0080_ID000283,run0087_ID000156,run0091_ID000232,run0103_ID000247,run0106_ID000232,run0108_ID000020,run0111_ID000118,run0117_ID000327,run0121_ID000209,run0122_ID000021,run0124_ID000071,run0125_ID000077,run0126_ID000344,run0131_ID000391,run0132_ID000380,run0150_ID000189,run0151_ID000312,run0156_ID000028,run0158_ID000359,run0159_ID000035,run0160_ID000149,run0162_ID000268,run0167_ID000041,run0170_ID000124,run0170_ID000218,run0170_ID000400,run0170_ID000418,run0175_ID000139,run0175_ID000300,run0176_ID000218,run0176_ID000272,run0177_ID000070,run0183_ID000331,run0187_ID000435,run0189_ID000293,run0191_ID000229,run0193_ID000013,run0195_ID000111,run0202_ID000245,run0204_ID000284,run0207_ID000154,run0209_ID000300,run0210_ID000016,run0210_ID000040,run0210_ID000221,run0211_ID000312,run0211_ID000462,run0216_ID000185,run0216_ID000333,run0219_ID000029,run0221_ID000373,run0222_ID000126,run0226_ID000442,run0228_ID000404,run0236_ID000181,run0240_ID000477,run0242_ID000208,run0245_ID000236,run0250_ID000440,run0251_ID000377,run0254_ID000090,run0254_ID000470,run0256_ID000112,run0263_ID000260,run0266_ID000397,run0270_ID000260,run0270_ID000262,run0271_ID000336,run0276_ID000003,run0276_ID000042,run0278_ID000111,run0279_ID000405,run0280_ID000101,run0285_ID000014,run0286_ID000206,run0290_ID000056,run0291_ID000193,run0292_ID000156,run0298_ID000339,run0302_ID000382,run0314_ID000223,run0321_ID000159,run0323_ID000052,run0339_ID000132,run0340_ID000247,run0342_ID000307,run0347_ID000361,run0350_ID000115,run0352_ID000021,run0352_ID000163,run0355_ID000275,run0356_ID000097,run0361_ID000216,run0368_ID000124,run0374_ID000020,run0378_ID000129,run0384_ID000378,run0389_ID000142,run0389_ID000280,run0396_ID000359,run0401_ID000227,run0410_ID000250,run0410_ID000273,run0415_ID000055,run0419_ID000300,run0420_ID000152,run0421_ID000208,run0421_ID000383,run0434_ID000189,run0445_ID000162,run0446_ID000308,run0454_ID000028,run0468_ID000072,run0473_ID000322,run0475_ID000120,run0481_ID000069,run0484_ID000321,run0486_ID000108,run0487_ID000320,run0489_ID000040,run0489_ID000339,run0489_ID000353,run0491_ID000173,run0492_ID000231,run0498_ID000168,run0503_ID000143,run0509_ID000243,run0512_ID000238,run0516_ID000336,run0522_ID000294,run0524_ID000087,run0529_ID000420,run0533_ID000144,run0535_ID000141,run0537_ID000149,run0543_ID000437,run0546_ID000406,run0550_ID000253,run0550_ID000494,run0551_ID000099,run0551_ID000150,run0557_ID000319,run0558_ID000096,run0564_ID000354,run0577_ID000106,run0593_ID000019,run0598_ID000199,run0603_ID000005,run0603_ID000195,run0606_ID000304,run0608_ID000286,run0610_ID000063,run0613_ID000309,run0613_ID000375,run0615_ID000245,run0617_ID000297,run0619_ID000308,run0627_ID000386,run0629_ID000087,run0635_ID000075,run0642_ID000185,run0642_ID000462,run0647_ID000398,run0653_ID000037,run0653_ID000078,run0658_ID000054,run0664_ID000064,run0664_ID000350,run0667_ID000302,run0668_ID000294,run0672_ID000265,run0677_ID000457,run0684_ID000011,run0684_ID000346,run0687_ID000058,run0688_ID000432,run0689_ID000454,run0689_ID000513,run0700_ID000206,run0710_ID000198,run0719_ID000018,run0720_ID000203,run0729_ID000124,run0731_ID000202,run0734_ID000309,run0735_ID000071,run0766_ID000057,run0782_ID000343,run0785_ID000315,run0787_ID000040,run0793_ID000454,run0798_ID000437,run0804_ID000264,run0808_ID000191,run0813_ID000124,run0822_ID000056,run0836_ID000198,run0836_ID000340,run0837_ID000263,run0842_ID000055,run0842_ID000212,run0842_ID000312,run0865_ID000336,run0869_ID000370,run0877_ID000060,run0877_ID000233,run0885_ID000163,run0889_ID000211,run0889_ID000364
0-25,0.845810,0.460389,0.973858,0.924569,0.164041,0.065891,1.087872,1.20

# Results

In [35]:
templates_detections = dotdict({})
for templ_det in lima_sigma_table:
    templates_detections[templ_det] = dotdict({}) 
    templates_detections[templ_det]["li_ma"] = dotdict({})
    templates_detections[templ_det]["cnn"] = dotdict({})
    templates_detections[templ_det]["rnn"] = dotdict({})

In [36]:
for templ_det in lima_sigma_table:
    detections_3s = lima_sigma_table[templ_det] >= 3
    detections_5s = lima_sigma_table[templ_det] >= 5
    templates_detections[templ_det]["li_ma"]["3s_detection"] = detections_3s.any()
    templates_detections[templ_det]["li_ma"]["5s_detection"] = detections_5s.any()
    templates_detections[templ_det]["li_ma"]["3s_detections_indexes"] = np.where(detections_3s == True)[0]
    templates_detections[templ_det]["li_ma"]["5s_detections_indexes"] = np.where(detections_5s == True)[0]

for templ_det in sigma_table_rnn:
    detections_3s = sigma_table_rnn[templ_det] >= 3
    detections_5s = sigma_table_rnn[templ_det] >= 5
    templates_detections[templ_det]["rnn"]["3s_detection"] = detections_3s.any()
    templates_detections[templ_det]["rnn"]["5s_detection"] = detections_5s.any()
    templates_detections[templ_det]["rnn"]["3s_detections_indexes"] = np.where(detections_3s == True)[0]
    templates_detections[templ_det]["rnn"]["5s_detections_indexes"] = np.where(detections_5s == True)[0]
    
for templ_det in sigma_table_cnn:
    detections_3s = sigma_table_cnn[templ_det] >= 3
    detections_5s = sigma_table_cnn[templ_det] >= 5
    templates_detections[templ_det]["cnn"]["3s_detection"] = detections_3s.any()
    templates_detections[templ_det]["cnn"]["5s_detection"] = detections_5s.any()
    templates_detections[templ_det]["cnn"]["3s_detections_indexes"] = np.where(detections_3s == True)[0]
    templates_detections[templ_det]["cnn"]["5s_detections_indexes"] = np.where(detections_5s == True)[0]    

In [37]:
def get_dd(model, templates_detections, onset_index=45):
    first_detection_indexes = []
    for template_result in templates_detections.values():
        if len(template_result[model]["3s_detections_indexes"]) > 0:
            first_detection_indexes.append(template_result[model]["3s_detections_indexes"][0])
    return round(np.array(first_detection_indexes).mean()-onset_index,2)

In [38]:
print(f'Detected by CNN: {np.array([template_result["cnn"]["3s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("cnn",templates_detections)} s')
print(f'Detected by RNN: {np.array([template_result["rnn"]["3s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("rnn",templates_detections)} s')
print(f'Detected by Li&Ma: {np.array([template_result["li_ma"]["3s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("li_ma",templates_detections)} s')

Detected by CNN: 16 - dd: 16.06 s
Detected by RNN: 65 - dd: 22.86 s
Detected by Li&Ma: 137 - dd: 12.24 s


In [39]:
print(f'Detected by CNN: {np.array([template_result["cnn"]["5s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("cnn",templates_detections)} s')
print(f'Detected by RNN: {np.array([template_result["rnn"]["5s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("rnn",templates_detections)} s')
print(f'Detected by Li&Ma: {np.array([template_result["li_ma"]["5s_detection"] for template_result in templates_detections.values()]).sum()} - dd: {get_dd("li_ma",templates_detections)} s')

Detected by CNN: 3 - dd: 16.06 s
Detected by RNN: 6 - dd: 22.86 s
Detected by Li&Ma: 46 - dd: 12.24 s


In [ ]:
matplotlib.use("TKAgg", force=True)
%matplotlib inline
from rtapipe.lib.plotting.PlotConfig import PlotConfig

def sigma_plot(template, sigma_table_model, lima_sigma_table):
    pc = PlotConfig()    
    fig, ax = plt.subplots(1,1, figsize=pc.fig_size)
    ax.plot(sigma_table_model.index.values, sigma_table_model[template], marker='o', linestyle='--', label="Anomaly Detection")
    ax.scatter(sigma_table_model.index.values, lima_sigma_table[template], marker='x', label="Li&Ma")
    ax.tick_params(axis='y', labelsize=15, width=2, grid_alpha=0.5)
    ax.tick_params(axis='x', which='major', labelsize=10)
    fig.suptitle("Detection plot")
    ax.set_title(f"Template {template}")
    ax.set_ylabel("Sigma")
    ax.set_xlabel("Timeseries")
    ax.set_ylim(0, 10)
    plt.axvline(x = 45, color = 'grey', linestyle="-.")
    plt.axhline(y = 3, color = 'orange', linestyle="-.")
    plt.axhline(y = 5, color = 'red', linestyle="-.")
    mask = [True if i%2==0 else False for i in range(len(sigma_table_model.index.values))]
    ax.set_xticks(range(0, len(sigma_table_model.index.values), 2), sigma_table_model.index.values[mask], rotation=45)
    ax.legend()
    fig.savefig(Path(output_dir).joinpath(f"detection_plot_{template}.png"), dpi=pc.dpi)
    plt.show()    
    
for template in tqdm(templates):
    sigma_plot(template, sigma_table_rnn, lima_sigma_table)